# Cost Optimization Analysis

This notebook focuses on practical cost optimization strategies and analysis techniques.

## Learning Objectives
- Compare costs across different LLM models
- Analyze token efficiency and optimization opportunities
- Evaluate cost-performance tradeoffs
- Calculate ROI for different optimization strategies
- Generate actionable recommendations

## Prerequisites
```bash
pip install pandas matplotlib seaborn numpy requests scikit-learn
```

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
from datetime import datetime, timedelta
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('Set2')
%matplotlib inline

## 1. Load Cost Data

In [ ]:
# API Configuration
BASE_URL = 'http://localhost:3000/api'
API_KEY = 'your-api-key-here'
HEADERS = {
    'Authorization': f'Bearer {API_KEY}',
    'Content-Type': 'application/json'
}

# Fetch data
end_date = datetime.now()
start_date = end_date - timedelta(days=30)

params = {
    'start_date': start_date.isoformat(),
    'end_date': end_date.isoformat(),
    'limit': 5000
}

response = requests.get(f'{BASE_URL}/cost-tracking', headers=HEADERS, params=params)
cost_data = response.json()

df = pd.DataFrame(cost_data['data'])
df['timestamp'] = pd.to_datetime(df['timestamp'])
df['total_tokens'] = df['input_tokens'] + df['output_tokens']
df['cost_per_1k_tokens'] = (df['total_cost'] / df['total_tokens'] * 1000).round(4)

print(f"Loaded {len(df)} cost records")
print(f"Date range: {df['timestamp'].min()} to {df['timestamp'].max()}")
print(f"Models: {df['model'].unique().tolist()}")

## 2. Model Cost Comparison

In [ ]:
# Comprehensive model comparison
model_comparison = df.groupby('model').agg({
    'total_cost': ['sum', 'mean', 'median', 'std'],
    'request_id': 'count',
    'total_tokens': 'sum',
    'input_tokens': 'sum',
    'output_tokens': 'sum',
    'cost_per_1k_tokens': 'mean'
}).round(4)

model_comparison.columns = ['_'.join(col).strip() for col in model_comparison.columns.values]
model_comparison = model_comparison.rename(columns={
    'total_cost_sum': 'total_cost',
    'total_cost_mean': 'avg_cost_per_request',
    'total_cost_median': 'median_cost_per_request',
    'total_cost_std': 'cost_std_dev',
    'request_id_count': 'request_count',
    'total_tokens_sum': 'total_tokens',
    'input_tokens_sum': 'input_tokens',
    'output_tokens_sum': 'output_tokens',
    'cost_per_1k_tokens_mean': 'avg_cost_per_1k_tokens'
})

# Calculate percentages
total_cost_all = model_comparison['total_cost'].sum()
model_comparison['cost_percentage'] = (model_comparison['total_cost'] / total_cost_all * 100).round(2)

# Sort by total cost
model_comparison = model_comparison.sort_values('total_cost', ascending=False)

print("Model Cost Comparison:")
print("="*100)
print(model_comparison[['total_cost', 'request_count', 'avg_cost_per_request', 
                         'avg_cost_per_1k_tokens', 'cost_percentage']])

In [ ]:
# Visualize model cost comparison
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Total cost by model
ax1 = axes[0, 0]
model_comparison['total_cost'].plot(kind='bar', ax=ax1, color='steelblue')
ax1.set_title('Total Cost by Model', fontsize=12, fontweight='bold')
ax1.set_ylabel('Total Cost ($)')
ax1.set_xlabel('Model')
ax1.tick_params(axis='x', rotation=45)
ax1.grid(True, alpha=0.3, axis='y')

# Average cost per request
ax2 = axes[0, 1]
model_comparison['avg_cost_per_request'].plot(kind='bar', ax=ax2, color='coral')
ax2.set_title('Average Cost per Request', fontsize=12, fontweight='bold')
ax2.set_ylabel('Avg Cost ($)')
ax2.set_xlabel('Model')
ax2.tick_params(axis='x', rotation=45)
ax2.grid(True, alpha=0.3, axis='y')

# Cost per 1K tokens
ax3 = axes[1, 0]
model_comparison['avg_cost_per_1k_tokens'].plot(kind='bar', ax=ax3, color='seagreen')
ax3.set_title('Average Cost per 1K Tokens', fontsize=12, fontweight='bold')
ax3.set_ylabel('Cost per 1K Tokens ($)')
ax3.set_xlabel('Model')
ax3.tick_params(axis='x', rotation=45)
ax3.grid(True, alpha=0.3, axis='y')

# Request distribution
ax4 = axes[1, 1]
model_comparison['request_count'].plot(kind='bar', ax=ax4, color='mediumpurple')
ax4.set_title('Request Count by Model', fontsize=12, fontweight='bold')
ax4.set_ylabel('Number of Requests')
ax4.set_xlabel('Model')
ax4.tick_params(axis='x', rotation=45)
ax4.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

## 3. Token Efficiency Analysis

In [ ]:
# Calculate token efficiency metrics
token_efficiency = df.groupby('model').apply(
    lambda x: pd.Series({
        'avg_input_tokens': x['input_tokens'].mean(),
        'avg_output_tokens': x['output_tokens'].mean(),
        'avg_total_tokens': x['total_tokens'].mean(),
        'input_output_ratio': x['input_tokens'].sum() / x['output_tokens'].sum() if x['output_tokens'].sum() > 0 else 0,
        'tokens_per_dollar': (x['total_tokens'].sum() / x['total_cost'].sum()) if x['total_cost'].sum() > 0 else 0
    })
).round(2)

print("Token Efficiency by Model:")
print("="*80)
print(token_efficiency)

In [ ]:
# Visualize token efficiency
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Input vs Output tokens
ax1 = axes[0]
x = np.arange(len(token_efficiency))
width = 0.35
ax1.bar(x - width/2, token_efficiency['avg_input_tokens'], width, label='Input Tokens', alpha=0.8)
ax1.bar(x + width/2, token_efficiency['avg_output_tokens'], width, label='Output Tokens', alpha=0.8)
ax1.set_title('Average Token Usage by Model', fontsize=12, fontweight='bold')
ax1.set_ylabel('Average Tokens per Request')
ax1.set_xlabel('Model')
ax1.set_xticks(x)
ax1.set_xticklabels(token_efficiency.index, rotation=45, ha='right')
ax1.legend()
ax1.grid(True, alpha=0.3, axis='y')

# Tokens per dollar (value)
ax2 = axes[1]
token_efficiency['tokens_per_dollar'].plot(kind='bar', ax=ax2, color='gold')
ax2.set_title('Token Value (Tokens per Dollar)', fontsize=12, fontweight='bold')
ax2.set_ylabel('Tokens per Dollar')
ax2.set_xlabel('Model')
ax2.tick_params(axis='x', rotation=45)
ax2.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

## 4. Cost-Performance Tradeoff Analysis

In [ ]:
# Create performance score (normalized combination of efficiency metrics)
scaler = MinMaxScaler()

performance_df = model_comparison[['avg_cost_per_request', 'avg_cost_per_1k_tokens']].copy()
performance_df['cost_score'] = scaler.fit_transform(1 / performance_df[['avg_cost_per_request']])
performance_df['efficiency_score'] = scaler.fit_transform(token_efficiency[['tokens_per_dollar']])
performance_df['overall_score'] = (performance_df['cost_score'] + performance_df['efficiency_score']) / 2

performance_df = performance_df.sort_values('overall_score', ascending=False)

print("Cost-Performance Analysis:")
print("="*80)
print(performance_df[['cost_score', 'efficiency_score', 'overall_score']].round(3))

In [ ]:
# Scatter plot: Cost vs Efficiency
fig, ax = plt.subplots(figsize=(12, 8))

for model in model_comparison.index:
    x = model_comparison.loc[model, 'avg_cost_per_1k_tokens']
    y = token_efficiency.loc[model, 'tokens_per_dollar']
    size = model_comparison.loc[model, 'request_count'] / 10
    
    ax.scatter(x, y, s=size, alpha=0.6, label=model)
    ax.annotate(model, (x, y), xytext=(5, 5), textcoords='offset points', fontsize=9)

ax.set_title('Cost-Performance Tradeoff (bubble size = request count)', 
             fontsize=14, fontweight='bold')
ax.set_xlabel('Average Cost per 1K Tokens ($)', fontsize=12)
ax.set_ylabel('Tokens per Dollar', fontsize=12)
ax.grid(True, alpha=0.3)
ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

plt.tight_layout()
plt.show()

## 5. Optimization Opportunities

In [ ]:
# Identify potential savings by switching models
print("OPTIMIZATION OPPORTUNITIES")
print("="*80)

# Find the most cost-effective model
best_model = performance_df['overall_score'].idxmax()
best_cost_per_1k = model_comparison.loc[best_model, 'avg_cost_per_1k_tokens']

print(f"\nMost Cost-Effective Model: {best_model}")
print(f"Cost per 1K tokens: ${best_cost_per_1k:.4f}")

# Calculate potential savings
print("\n" + "-"*80)
print("Potential Savings by Switching to Most Cost-Effective Model:")
print("-"*80)

total_potential_savings = 0

for model in model_comparison.index:
    if model != best_model:
        current_cost = model_comparison.loc[model, 'total_cost']
        total_tokens = model_comparison.loc[model, 'total_tokens']
        potential_cost = (total_tokens / 1000) * best_cost_per_1k
        savings = current_cost - potential_cost
        savings_pct = (savings / current_cost * 100) if current_cost > 0 else 0
        
        if savings > 0:
            total_potential_savings += savings
            print(f"\n{model}:")
            print(f"  Current Cost: ${current_cost:.2f}")
            print(f"  Potential Cost with {best_model}: ${potential_cost:.2f}")
            print(f"  Potential Savings: ${savings:.2f} ({savings_pct:.1f}%)")

print(f"\n{'='*80}")
print(f"TOTAL POTENTIAL SAVINGS: ${total_potential_savings:.2f}")
print(f"Current Total Cost: ${model_comparison['total_cost'].sum():.2f}")
print(f"Potential Total Cost: ${model_comparison['total_cost'].sum() - total_potential_savings:.2f}")
print(f"Overall Savings: {(total_potential_savings / model_comparison['total_cost'].sum() * 100):.1f}%")
print("="*80)

In [ ]:
# Token usage optimization opportunities
print("\nTOKEN USAGE OPTIMIZATION OPPORTUNITIES")
print("="*80)

# Find requests with unusually high token usage
token_percentile_95 = df['total_tokens'].quantile(0.95)
high_token_requests = df[df['total_tokens'] > token_percentile_95]

print(f"\nHigh Token Usage Requests (>95th percentile, {token_percentile_95:.0f} tokens):")
print(f"Count: {len(high_token_requests)}")
print(f"Total Cost: ${high_token_requests['total_cost'].sum():.2f}")
print(f"Average Cost: ${high_token_requests['total_cost'].mean():.4f}")
print(f"\nBreakdown by Model:")
print(high_token_requests.groupby('model').agg({
    'request_id': 'count',
    'total_cost': 'sum',
    'total_tokens': 'mean'
}).round(2))

## 6. ROI Calculation for Optimization Strategies

In [ ]:
# Calculate ROI for different optimization strategies
print("ROI ANALYSIS FOR OPTIMIZATION STRATEGIES")
print("="*80)

# Strategy 1: Switch to most cost-effective model
print("\nStrategy 1: Switch all requests to most cost-effective model")
print("-"*80)
implementation_cost_1 = 500  # Estimated implementation cost
monthly_savings_1 = total_potential_savings
roi_months_1 = implementation_cost_1 / monthly_savings_1 if monthly_savings_1 > 0 else float('inf')
annual_savings_1 = monthly_savings_1 * 12

print(f"Implementation Cost: ${implementation_cost_1:.2f}")
print(f"Monthly Savings: ${monthly_savings_1:.2f}")
print(f"Annual Savings: ${annual_savings_1:.2f}")
print(f"ROI Period: {roi_months_1:.1f} months")
print(f"12-Month ROI: {((annual_savings_1 - implementation_cost_1) / implementation_cost_1 * 100):.1f}%")

# Strategy 2: Implement prompt optimization
print("\n\nStrategy 2: Implement prompt optimization (reduce tokens by 20%)")
print("-"*80)
implementation_cost_2 = 2000  # Higher cost for prompt engineering
token_reduction = 0.20
monthly_savings_2 = model_comparison['total_cost'].sum() * token_reduction
roi_months_2 = implementation_cost_2 / monthly_savings_2 if monthly_savings_2 > 0 else float('inf')
annual_savings_2 = monthly_savings_2 * 12

print(f"Implementation Cost: ${implementation_cost_2:.2f}")
print(f"Expected Token Reduction: {token_reduction*100:.0f}%")
print(f"Monthly Savings: ${monthly_savings_2:.2f}")
print(f"Annual Savings: ${annual_savings_2:.2f}")
print(f"ROI Period: {roi_months_2:.1f} months")
print(f"12-Month ROI: {((annual_savings_2 - implementation_cost_2) / implementation_cost_2 * 100):.1f}%")

# Strategy 3: Implement caching
print("\n\nStrategy 3: Implement response caching (30% cache hit rate)")
print("-"*80)
implementation_cost_3 = 1500
cache_hit_rate = 0.30
monthly_savings_3 = model_comparison['total_cost'].sum() * cache_hit_rate
roi_months_3 = implementation_cost_3 / monthly_savings_3 if monthly_savings_3 > 0 else float('inf')
annual_savings_3 = monthly_savings_3 * 12

print(f"Implementation Cost: ${implementation_cost_3:.2f}")
print(f"Expected Cache Hit Rate: {cache_hit_rate*100:.0f}%")
print(f"Monthly Savings: ${monthly_savings_3:.2f}")
print(f"Annual Savings: ${annual_savings_3:.2f}")
print(f"ROI Period: {roi_months_3:.1f} months")
print(f"12-Month ROI: {((annual_savings_3 - implementation_cost_3) / implementation_cost_3 * 100):.1f}%")

In [ ]:
# Visualize ROI comparison
strategies = [
    {'name': 'Model Switch', 'cost': implementation_cost_1, 'monthly_savings': monthly_savings_1},
    {'name': 'Prompt Optimization', 'cost': implementation_cost_2, 'monthly_savings': monthly_savings_2},
    {'name': 'Response Caching', 'cost': implementation_cost_3, 'monthly_savings': monthly_savings_3}
]

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Monthly savings comparison
ax1 = axes[0]
names = [s['name'] for s in strategies]
savings = [s['monthly_savings'] for s in strategies]
ax1.bar(names, savings, color=['steelblue', 'coral', 'seagreen'], alpha=0.8)
ax1.set_title('Monthly Savings by Strategy', fontsize=12, fontweight='bold')
ax1.set_ylabel('Monthly Savings ($)')
ax1.tick_params(axis='x', rotation=15)
ax1.grid(True, alpha=0.3, axis='y')

# ROI timeline
ax2 = axes[1]
months = np.arange(0, 13)
for strategy in strategies:
    net_benefit = months * strategy['monthly_savings'] - strategy['cost']
    ax2.plot(months, net_benefit, marker='o', label=strategy['name'], linewidth=2)

ax2.axhline(y=0, color='black', linestyle='--', linewidth=1)
ax2.set_title('ROI Timeline (12 Months)', fontsize=12, fontweight='bold')
ax2.set_xlabel('Months')
ax2.set_ylabel('Net Benefit ($)')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 7. Optimization Recommendations

In [ ]:
# Generate prioritized recommendations
print("\n" + "="*80)
print("PRIORITIZED OPTIMIZATION RECOMMENDATIONS")
print("="*80)

recommendations = []

# Recommendation 1: Model optimization
if total_potential_savings > 0:
    recommendations.append({
        'priority': 'HIGH',
        'title': f'Switch to {best_model} for cost savings',
        'impact': f'${total_potential_savings:.2f}/month',
        'effort': 'Low',
        'description': f'Migrating appropriate workloads to {best_model} could save {(total_potential_savings / model_comparison["total_cost"].sum() * 100):.1f}% of total costs.'
    })

# Recommendation 2: High token usage
if len(high_token_requests) > 0:
    high_token_cost = high_token_requests['total_cost'].sum()
    high_token_pct = len(high_token_requests) / len(df) * 100
    recommendations.append({
        'priority': 'HIGH',
        'title': 'Optimize high-token requests',
        'impact': f'Potential ${high_token_cost * 0.3:.2f}/month (30% reduction)',
        'effort': 'Medium',
        'description': f'{high_token_pct:.1f}% of requests use excessive tokens. Implement prompt optimization and input validation.'
    })

# Recommendation 3: Caching
recommendations.append({
    'priority': 'MEDIUM',
    'title': 'Implement response caching',
    'impact': f'${monthly_savings_3:.2f}/month',
    'effort': 'Medium',
    'description': 'With an estimated 30% cache hit rate, caching could significantly reduce costs.'
})

# Recommendation 4: Rate limiting during peak hours
recommendations.append({
    'priority': 'LOW',
    'title': 'Implement smart rate limiting',
    'effort': 'Low',
    'impact': 'Variable',
    'description': 'Implement request batching and rate limiting to optimize API usage patterns.'
})

# Print recommendations
for i, rec in enumerate(recommendations, 1):
    print(f"\n{i}. [{rec['priority']} PRIORITY] {rec['title']}")
    print(f"   Impact: {rec['impact']}")
    print(f"   Effort: {rec['effort']}")
    print(f"   Details: {rec['description']}")

print("\n" + "="*80)

## 8. Create Optimization Action Plan

In [ ]:
# Create action plan DataFrame
action_plan = pd.DataFrame(recommendations)
action_plan['priority_order'] = action_plan['priority'].map({'HIGH': 1, 'MEDIUM': 2, 'LOW': 3})
action_plan = action_plan.sort_values('priority_order')

print("\nOPTIMIZATION ACTION PLAN")
print("="*80)
print(action_plan[['priority', 'title', 'impact', 'effort']].to_string(index=False))

# Calculate total potential impact
total_impact = monthly_savings_1 + (high_token_requests['total_cost'].sum() * 0.3) + monthly_savings_3
print(f"\n\nEstimated Total Monthly Impact: ${total_impact:.2f}")
print(f"Estimated Annual Impact: ${total_impact * 12:.2f}")
print(f"Current Monthly Cost: ${model_comparison['total_cost'].sum():.2f}")
print(f"Optimized Monthly Cost: ${model_comparison['total_cost'].sum() - total_impact:.2f}")
print(f"Total Potential Savings: {(total_impact / model_comparison['total_cost'].sum() * 100):.1f}%")

## 9. Export Optimization Report

In [ ]:
# Export results
output_dir = 'cost_optimization_output'
import os
os.makedirs(output_dir, exist_ok=True)

# Export model comparison
model_comparison.to_csv(f'{output_dir}/model_comparison.csv')
print(f"Saved: {output_dir}/model_comparison.csv")

# Export token efficiency
token_efficiency.to_csv(f'{output_dir}/token_efficiency.csv')
print(f"Saved: {output_dir}/token_efficiency.csv")

# Export performance analysis
performance_df.to_csv(f'{output_dir}/performance_analysis.csv')
print(f"Saved: {output_dir}/performance_analysis.csv")

# Export high token requests
high_token_requests.to_csv(f'{output_dir}/high_token_requests.csv', index=False)
print(f"Saved: {output_dir}/high_token_requests.csv")

# Export action plan
action_plan[['priority', 'title', 'impact', 'effort', 'description']].to_csv(
    f'{output_dir}/action_plan.csv', index=False
)
print(f"Saved: {output_dir}/action_plan.csv")

print(f"\nAll optimization reports exported to '{output_dir}/' directory")

## Summary

In this notebook, you learned:
- How to compare costs across different LLM models
- Techniques for analyzing token efficiency
- Methods for evaluating cost-performance tradeoffs
- ROI calculations for optimization strategies
- How to generate actionable recommendations

## Next Steps
- **04_custom_reports.ipynb** - Build custom reporting dashboards
- **05_ml_forecasting.ipynb** - Advanced ML forecasting techniques
- Implement the recommended optimizations
- Monitor results and iterate